In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('./preprocessing')
sys.path.append('./seq2seq')

In [3]:
from processor import Code_Intent_Pairs
from model import Seq2Seq
from data import get_test_loader

### Get Data Loader

In [4]:
code_intent_pair = Code_Intent_Pairs()

In [5]:
path = 'vocab/'
code_intent_pair.load_dict(path)
special_symbols = code_intent_pair.get_special_symbols()
word_size = code_intent_pair.get_word_size()
code_size = code_intent_pair.get_code_size()

In [6]:
test_path = 'processed_corpus/test.json'
test_entries = code_intent_pair.load_entries(test_path)

In [7]:
testloader = get_test_loader(test_entries)

### Get Model

In [8]:
hyperP = {
    ## training parameters
    'batch_size' : 32,
    'lr' : 1e-3,
    'teacher_force_rate' : 0.90,
    'max_epochs' : 50,
    'lr_keep_rate' : 0.95,  # set to 1.0 to not decrease lr overtime
    'load_pretrain_code_embed': False,
    'freeze_embed': False,
    
    ## encoder architecture
    'encoder_layers' : 2,
    'encoder_embed_size' : 128,
    'encoder_hidden_size' : 384,
    'encoder_dropout_rate' : 0.3,
    
    ## decoder architecture
    'decoder_layers' : 2,
    'decoder_embed_size' : 128,
    'decoder_hidden_size' : 384,
    'decoder_dropout_rate' : 0.3,
    
    ## attn architecture
    'attn_hidden_size' : 384,
    
    ## visualization
    'print_every': 10,
}

In [9]:
model = Seq2Seq(word_size, code_size, hyperP)

In [10]:
import torch
if hyperP['load_pretrain_code_embed']:
    model.decoder.embed[0].load_state_dict(torch.load('./pretrain_code_lm/embedding-1556211835.t7'))
    if hyperP['freeze_embed']:
        for param in model.decoder.embed[0].parameters():
            param.requires_grad = False

In [11]:
model.load('model_30.t7')

### Test Decoding

In [12]:
from decoder import Decoder
from decoder import post_process_dummy, post_process_pmi
from evaluate import get_bleu_all, get_bleu_sent

In [13]:
beam_decoder = Decoder(model)

In [14]:
model.eval()
sos = special_symbols['code_sos']
eos = special_symbols['code_eos']
unk = special_symbols['code_unk']

In [15]:
idx2code = code_intent_pair.idx2code

In [16]:
intent2idx = code_intent_pair.intent2idx

#### Beam Search Results

In [17]:
dummy_code_list = []
true_code_list = []

for i, (src_seq, slot_map, code, intent) in enumerate(testloader):
    beams = beam_decoder.decode(src_seq, sos, eos, unk, beam_width=3)
    dummy_code =  post_process_dummy(slot_map, beams, idx2code)
    dummy_code_list.append(dummy_code)
    true_code_list.append(code)

In [18]:
get_bleu_all(dummy_code_list, true_code_list)

0.2726735512450424

#### Rerank with PMI

In [30]:
import pickle
with open('pmi_matrix', 'rb') as f:
    pmi = pickle.load(f)

In [24]:
from processor import process_intent

pmi_code_list = []
true_code_list = []

for i, (src_seq, slot_map, code, intent) in enumerate(testloader):

    beams = beam_decoder.decode(src_seq, sos, eos, unk, beam_width=3)
    pmi_code =  post_process_pmi(intent, beams, 
                                 idx2code, intent2idx, pmi, process_intent)
    
    pmi_code_list.append(pmi_code)
    true_code_list.append(code)

In [25]:
get_bleu_all(pmi_code_list, true_code_list)

0.2891773196832087

In [31]:
from processor import process_intent

pmi_code_list = []
true_code_list = []

for i, (src_seq, slot_map, code, intent) in enumerate(testloader):

    beams = beam_decoder.decode(src_seq, sos, eos, unk, beam_width=20)
    pmi_code =  post_process_pmi(intent, beams, 
                                 idx2code, intent2idx, pmi, process_intent)
    
    pmi_code_list.append(pmi_code)
    true_code_list.append(code)

In [32]:
get_bleu_all(pmi_code_list, true_code_list)

0.32096633902279686

In [34]:
for i, (src_seq, slot_map, code, intent) in enumerate(testloader):
    print('intent:     \t', intent)
    print('groud trugh:\t', code)
    print('prediction: \t', pmi_code_list[i])
    print()
    
    if i == 100:
        break

intent:     	 send a signal `signal.SIGUSR1` to the current process
groud trugh:	 os.kill(os.getpid(), signal.SIGUSR1)
prediction: 	 try : 
      print ( ) 
  except : 
      print ( 'signal.SIGUSR1' ) 
  except : 
      print ( ) ) 
  except : . write ( )

intent:     	 decode a hex string '4a4b4c' to UTF-8.
groud trugh:	 bytes.fromhex('4a4b4c').decode('utf-8')
prediction: 	 """4a4b4c""" . encode ( 'hex' ) . decode ( 'utf8' )

intent:     	 check if all elements in list `myList` are identical
groud trugh:	 all(x == myList[0] for x in myList)
prediction: 	 all ( isinstance ( word , myList ) ) == 1

intent:     	 format number of spaces between strings `Python`, `:` and `Very Good` to be `20`
groud trugh:	 print('%*s : %*s' % (20, 'Python', 20, 'Very Good'))
prediction: 	 print ( re . sub ( '(:)' , 'Very Good' , Python ) )

intent:     	 How to convert a string from CP-1251 to UTF-8?
groud trugh:	 d.decode('cp1251').encode('utf8')
prediction: 	 s . format . encode ( ) . decode ( 'utf-8'

#### Test

In [249]:
from decoder import post_process_test

src_seq, slot_map, code, intent = testloader[43]

beams = beam_decoder.decode(src_seq, sos, eos, unk, beam_width=5)
post_process_test(intent, beams, idx2code, intent2idx, pmi, process_intent, code)

intent:      	 convert hex string '470FC614' to a float number
ground truth:	 struct.unpack('!f', '470FC614'.decode('hex'))[0]
before rerank:
Model score:-0.33	BLEU score:0.31:	struct . unpack ( 'd' , struct . pack ( 'hex' , 16 ) )
Model score:-0.30	BLEU score:0.08:	hex ( int ( '470FC614' , 16 ) )
Model score:-0.28	BLEU score:0.08:	int ( int ( '470FC614' , 16 ) )
Model score:-0.26	BLEU score:0.08:	int ( ord ( '470FC614' , 16 ) )
Model score:-0.15	BLEU score:0.04:	int ( '470FC614' , 16 )

after rerank:
Rerank score:1.14	BLEU score:0.08:	int ( int ( '470FC614' , 16 ) )
Rerank score:1.33	BLEU score:0.04:	int ( '470FC614' , 16 )
Rerank score:1.42	BLEU score:0.08:	int ( ord ( '470FC614' , 16 ) )
Rerank score:1.59	BLEU score:0.08:	hex ( int ( '470FC614' , 16 ) )
Rerank score:2.01	BLEU score:0.31:	struct . unpack ( 'd' , struct . pack ( 'hex' , 16 ) )


#### Rerank with Neural Model

In [47]:
from rerank_model import ScoreNet

In [48]:
hyperP = {
    ## encoder architecture
    'encoder_layers': 2,
    'encoder_embed_size': 128,
    'encoder_hidden_size': 256,
    'encoder_dropout_rate': 0.3,
}

In [49]:
score_net = ScoreNet(word_size, code_size, hyperP).cuda()

In [50]:
score_net.load()

In [51]:
from processor import process_intent

model_code_list = []
true_code_list = []

for i, (src_seq, _, code, intent) in enumerate(testloader):
    beams = beam_decoder.decode(src_seq, sos, eos, unk, beam_width=20)
    model_code =  post_process_model(intent, beams, idx2code, score_net, process_intent, intent2idx)
    model_code_list.append(model_code)
    true_code_list.append(code)

In [39]:
get_bleu_all(pmi_code_list, true_code_list)

0.3189677547378042

#### Write Hand Featured Rerank

In [111]:
from data import write_answer_json
write_answer_json(pmi_code_list)

In [112]:
!zip answer.zip answer.txt

updating: answer.txt (deflated 72%)
